# Classification 10/20/22

Goal: Start classifying tidydata_1020222 using sktime module, specificaly classify based on differences in temp / rh 



In [2]:
import json
import datetime
import pandas as pd
import numpy as np
import sys
import os
import plotly.express as px
from sktime.classification.kernel_based import RocketClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
df = pd.read_csv("../constants/td_ambient_102022.csv", )
df

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1
...,...,...,...,...,...,...,...
123601,2022-09-11 07:58:00,29.196,50.861,1,19.611111,30.000000,0
123602,2022-09-11 07:58:30,29.196,50.861,1,19.611111,30.000000,0
123603,2022-09-11 07:59:00,29.196,50.796,1,19.611111,30.000000,0
123604,2022-09-11 07:59:30,29.196,50.796,1,19.611111,30.000000,0


In [6]:
# add temperature and rh deltas for each observation 
df["T_Delta"] = df["Ambient Temp"] - df["Temp C"]
df["RH_Delta"] = df["Ambient RH"] - df["RH %"]

In [7]:
df

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open,T_Delta,RH_Delta
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1,-8.855667,-20.712444
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1,-8.855667,-20.806444
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1,-8.855667,-20.932444
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1,-8.855667,-20.901444
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1,-8.831667,-20.961444
...,...,...,...,...,...,...,...,...,...
123601,2022-09-11 07:58:00,29.196,50.861,1,19.611111,30.000000,0,-9.584889,-20.861000
123602,2022-09-11 07:58:30,29.196,50.861,1,19.611111,30.000000,0,-9.584889,-20.861000
123603,2022-09-11 07:59:00,29.196,50.796,1,19.611111,30.000000,0,-9.584889,-20.796000
123604,2022-09-11 07:59:30,29.196,50.796,1,19.611111,30.000000,0,-9.584889,-20.796000


In [25]:
# do some grouping 

# group by window open vs window closed -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Window Open"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
                 mean       std       mean       std
Window Open                                         
0           -5.752171  4.206669 -30.640394  8.547424
1           -6.257008  3.594757 -28.279236  7.846737

In [24]:
# group by rooms -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Room"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
          mean       std       mean       std
Room                                         
0    -5.729435  3.780207 -29.178837  8.143704
1    -6.301897  4.010265 -29.637181  8.395411

In [26]:
# group by window open vs window closed and rooms -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Window Open", "Room"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
                      mean       std       mean       std
Window Open Room                                         
0           0    -1.603624  4.786734 -37.193811  7.443892
            1    -6.059046  3.992906 -30.155626  8.423356
1           0    -6.020292  3.521054 -28.613806  7.889349
            1    -8.272099  3.584829 -25.431145  6.841461

In [27]:
# df.drop("DateTime", axis=1).groupby(["Room", "Window Open"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]
# switching the grouping order doesnt change values of numers

T_Delta             RH_Delta          
                      mean       std       mean       std
Room Window Open                                         
0    0           -1.603624  4.786734 -37.193811  7.443892
     1           -6.020292  3.521054 -28.613806  7.889349
1    0           -6.059046  3.992906 -30.155626  8.423356
     1           -8.272099  3.584829 -25.431145  6.841461

# Split into train / test data

In [32]:
x = df.iloc[:, 1:4]
x

,Temp C,RH %,Room
0,23.189,51.268,0
1,23.189,51.362,0
2,23.189,51.488,0
3,23.189,51.457,0
4,23.165,51.517,0
...,...,...,...
123601,29.196,50.861,1
123602,29.196,50.861,1
123603,29.196,50.796,1
123604,29.196,50.796,1


In [33]:
x = x.to_numpy()
x

array([[23.189, 51.268,  0.   ],
       [23.189, 51.362,  0.   ],
       [23.189, 51.488,  0.   ],
       ...,
       [29.196, 50.796,  1.   ],
       [29.196, 50.796,  1.   ],
       [29.196, 50.796,  1.   ]])

In [34]:
y = df["Window Open"]
y = y.to_numpy()
y

array([1, 1, 1, ..., 0, 0, 0])

In [35]:

X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=42)

In [36]:
X_train

array([[24.731, 52.678,  1.   ],
       [29.897, 48.431,  1.   ],
       [24.103, 57.157,  1.   ],
       ...,
       [24.997, 49.381,  0.   ],
       [25.288, 51.742,  1.   ],
       [29.872, 49.011,  1.   ]])

In [37]:
y_test

array([0, 0, 0, ..., 1, 0, 0])

In [38]:
# data tooo large -> 82 mins running and still not done 
# rocket = RocketClassifier()
# rocket.fit(X_train, y_train)
# y_pred = rocket.predict(X_test)
# accuracy_score(y_test, y_pred)

# try to sample the data instead....